# Data lab 4 - Portfolio Performance Attribution and Factor Model

#### Note 1: Review what you have learned in Data labs, DataCamp assignments, and in-class sample code.
#### Note 2: Chapter "Performance Attribution" of DataCamp course "Introduction to Portfolio Analysis in Python"  is a useful reference.
#### Note 3: This note serves as a guide. You are free to tinker with it! 

### 1. Revisit the all-weather portfolio you crafted. Create the maximum Sharpe portfolio's daily return dataframe and then merge it with Fama French's five return factors.

#### If you have attempted the optional bonus, why not include the portfolios with L2 regularization and Black-Litterman model too.

In [67]:
# Import the necessary packages
import yfinance as yf
import datetime as dt 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [21]:
# Pick a list of stocks that form your all-weather portfolio
symbols_list = ["AAPL", "PG", "NEE", "UNH", "UNP", "INFY", "COST", "MDT", "AMT", "V"]
# Draw data for the past five years
start = dt.datetime(2017,9,1)
end = dt.datetime(2022,9,1)
data = yf.download(symbols_list, start=start, end=end)
#Illustrative, change into yours!
data.head(5)




[*********************100%***********************]  10 of 10 completed


Adj Close                                                          \
                 AAPL         AMT        COST      INFY        MDT        NEE   
Date                                                                            
2017-08-31  38.911678  133.133499  146.150085  6.548218  71.802490  33.610607   
2017-09-01  38.923538  131.083237  147.548721  6.478369  71.294830  33.532455   
2017-09-05  38.456127  132.135345  148.387909  6.395426  71.089973  33.554787   
2017-09-06  38.415794  131.353043  148.126816  6.382329  71.134521  33.105927   
2017-09-07  38.261566  132.180344  148.518478  6.364868  71.205765  33.382843   

                                                          ...     Volume  \
                   PG         UNH        UNP           V  ...       AAPL   
Date                                                      ...              
2017-08-31  80.494644  184.112137  94.792671  100.222595  ...  107140400   
2017-09-01  80.721443  184.898911  94.945686  100.590508  ...   66364400   
2017-09-05  80.887222  184.491669  93.712402   99.728851  ...  117874000   
2017-09-06  80.887222  183.630829  94.531609   99.893440  ...   86606800   
2017-09-07  81.105316  184.132553  94.963715  101.229477  ...   87714000   

                                                                            \
                AMT     COST      INFY      MDT      NEE       PG      UNH   
Date                                                                         
2017-08-31  2293200  4567700  17592000  5661900  6826000  5864400  2990200   
2017-09-01  1765800  2833500  10663800  3133400  4287200  4960700  2241800   
2017-09-05  1559300  3413000  12538600  3255800  4299200  6532100  2700800   
2017-09-06  1380700  2873700  12556600  3784000  9138400  6747200  3040200   
2017-09-07  1757400  2479300   8000400  3387000  7742400  7735400  2684600   

                               
                UNP         V  
Date                           
2017-08-31  2744400  13766000  
2017-09-01  2347700   4466800  
2017-09-05  2598700   6337500  
2017-09-06  4378700   6372200  
2017-09-07  3622900   6508600  

[5 rows x 60 columns]

In [22]:
# Keep only the adjusted close in the dataframe
# Note that the date is in the index
price = data["Adj Close"]

In [23]:
# Calculate return using method pct_change
# Find out more about .pct_change with help! 
daily_return = price.pct_change()
daily_return.head(5)

,AAPL,AMT,COST,INFY,MDT,NEE,PG,UNH,UNP,V
Date,,,,,,,,,,
2017-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-09-01,0.000305,-0.015400,0.009570,-0.010667,-0.007070,-0.002325,0.002818,0.004273,0.001614,0.003671
2017-09-05,-0.012008,0.008026,0.005688,-0.012803,-0.002873,0.000666,0.002054,-0.002203,-0.012989,-0.008566
2017-09-06,-0.001049,-0.005920,-0.001760,-0.002048,0.000627,-0.013377,0.000000,-0.004666,0.008742,0.001650
2017-09-07,-0.004015,0.006298,0.002644,-0.002736,0.001002,0.008365,0.002696,0.002732,0.004571,0.013375


In [24]:
# function that takes portfolio weights and creates a time-series of daily portfolio returns
def portfolio_return_series(daily_returns, weights):
    
    ''' 
    INPUTS
    daily returns: dataframe of daily returns. Each ticker column contains the series of daily returns for the ticker
    weights: numpy array of the portfolio weight on each ticker (sorted in ascending order)
    
    OUTPUTS
    portfolio_daily_returns: the portfolio return series given the weights
    
    '''    
    
    # Create portfolio daily returns
    portfolio_daily_returns = daily_returns.dot(weights)
    return portfolio_daily_returns

In [25]:
## Install PyPortfolioOpt package
# !pip install PyPortfolioOpt

# Import the packages 
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

The Maximum Sharpe Portfolio

In [26]:
# Calculate expected returns mu 
# Calculate expected returns mu 
mu = expected_returns.mean_historical_return(price)

# Calculate the covariance matrix S
cov_matrix_d = daily_return.cov()

# Obtain the efficient frontier
ef = EfficientFrontier(mu, cov_matrix_d)

# Calculate weights for the maximum Sharpe ratio portfolio
raw_weight_maxsharpe = ef.max_sharpe()
clean_weight_maxsharpe = ef.clean_weights()
# Inspect the calculated weights
print("Raw weight: ", raw_weight_maxsharpe)
print("Cleaned weight: ", clean_weight_maxsharpe)


Raw weight:  OrderedDict([('AAPL', 0.1739137370220427), ('AMT', 0.0), ('COST', 0.5956974006016518), ('INFY', 0.1138868143320729), ('MDT', 0.0), ('NEE', 0.0974204102023902), ('PG', 0.0), ('UNH', 0.0190816378418425), ('UNP', 0.0), ('V', 0.0)])
Cleaned weight:  OrderedDict([('AAPL', 0.17391), ('AMT', 0.0), ('COST', 0.5957), ('INFY', 0.11389), ('MDT', 0.0), ('NEE', 0.09742), ('PG', 0.0), ('UNH', 0.01908), ('UNP', 0.0), ('V', 0.0)])


The minimum volatility portfolio

In [27]:
# Obtain the efficient frontier
mu = expected_returns.mean_historical_return(price)
sigma = risk_models.sample_cov(price)
ef = EfficientFrontier(mu, sigma)
# Calculate weights for the minimum volatility portfolio
raw_weight_minvol = ef.min_volatility()
clean_weight_minvol = ef.clean_weights()
print("Raw weight: ", raw_weight_minvol)
print("Cleaned weight: ", clean_weight_minvol)

Raw weight:  OrderedDict([('AAPL', 0.0), ('AMT', 0.0095739449526793), ('COST', 0.2400400437637469), ('INFY', 0.1156782461562846), ('MDT', 0.160972342245783), ('NEE', 0.0739521879167142), ('PG', 0.3411637834496354), ('UNH', 0.0), ('UNP', 0.0586194515151567), ('V', 0.0)])
Cleaned weight:  OrderedDict([('AAPL', 0.0), ('AMT', 0.00957), ('COST', 0.24004), ('INFY', 0.11568), ('MDT', 0.16097), ('NEE', 0.07395), ('PG', 0.34116), ('UNH', 0.0), ('UNP', 0.05862), ('V', 0.0)])


In [28]:
# 1. Daily portfolio returns for the equally-weighted portfolio
equal_weight = np.repeat(0.1, 10)
equal_weight_return = portfolio_return_series(daily_return, equal_weight)
# Extract the first element from the function output for daily returns
equal_weight_first_element = equal_weight_return.iloc[1]

# 2. Daily portfolio returns for the maximum Sharpe portfolio
# Extract the first element from the function output for daily returns
clean_weight_maxsharpe_list = list(clean_weight_maxsharpe.values())
clean_weight_maxsharpe_array = np.array(clean_weight_maxsharpe_list)
max_sharpe_daily_return = portfolio_return_series(daily_return, clean_weight_maxsharpe_array)
max_sharpe_first_element = max_sharpe_daily_return.iloc[1]

# 3. Daily portfolio returns for the minimum volatility portfolio
# Extract the first element from the function output for daily returns
clean_weight_minvol_list = list(clean_weight_minvol.values())
clean_weight_minvol_array = np.array(clean_weight_minvol_list)
minvol_daily_return = portfolio_return_series(daily_return, clean_weight_minvol_array)
minvol_first_element = minvol_daily_return.iloc[1]



# Merge the three series side-by-side into a dataframe
# Note the index is date
portfolio_returns = pd.concat([equal_weight_return, max_sharpe_daily_return, minvol_daily_return], axis = 1)
# Rename column names
portfolio_returns = portfolio_returns.rename(columns = {0: "Equal_Weight_Return", 1: "MaxSharpe_Return", 2: "Minvol_Return"})
print(portfolio_returns)


            Equal_Weight_Return  MaxSharpe_Return  Minvol_Return
Date                                                            
2017-08-31                  NaN               NaN            NaN
2017-09-01            -0.001321          0.004394       0.000662
2017-09-05            -0.003501         -0.000136      -0.000513
2017-09-06            -0.001780         -0.002856      -0.001092
2017-09-07             0.003493          0.001432       0.002346
...                         ...               ...            ...
2022-08-25             0.010632          0.011389       0.008686
2022-08-26            -0.028264         -0.032142      -0.027499
2022-08-29            -0.004806         -0.005425      -0.004553
2022-08-30            -0.013817         -0.011904      -0.012361
2022-08-31            -0.005800         -0.005827      -0.006683

[1259 rows x 3 columns]


In [29]:
# Inspect the last five observations of the portfolio_returns dataframe
portfolio_returns.tail(5)

,Equal_Weight_Return,MaxSharpe_Return,Minvol_Return
Date,,,
2022-08-25,0.010632,0.011389,0.008686
2022-08-26,-0.028264,-0.032142,-0.027499
2022-08-29,-0.004806,-0.005425,-0.004553
2022-08-30,-0.013817,-0.011904,-0.012361
2022-08-31,-0.005800,-0.005827,-0.006683


In [52]:
# Read the csv file with factor returns with pd.read_csv()
# Note: place the file where your Jupyter notebook is
factor_returns = pd.read_csv("F-F_Research_Data_5_Factors_2x3_daily.csv")

# Divide all factor returns by 100
# Consistent with how we calculate portfolio returns
factor_returns.iloc[:, 1:] = (factor_returns.iloc[:, 1:]).div(100)




           Date  Mkt-RF     SMB     HML     RMW     CMA       RF
0      19630701 -0.0067  0.0002 -0.0035  0.0003  0.0013  0.00012
1      19630702  0.0079 -0.0028  0.0028 -0.0008 -0.0021  0.00012
2      19630703  0.0063 -0.0018 -0.0010  0.0013 -0.0025  0.00012
3      19630705  0.0040  0.0009 -0.0028  0.0007 -0.0030  0.00012
4      19630708 -0.0063  0.0007 -0.0020 -0.0027  0.0006  0.00012
...         ...     ...     ...     ...     ...     ...      ...
14890  20220825  0.0145  0.0014 -0.0001  0.0012 -0.0041  0.00008
14891  20220826 -0.0338 -0.0028  0.0169  0.0024  0.0087  0.00008
14892  20220829 -0.0072 -0.0038  0.0042  0.0023  0.0042  0.00008
14893  20220830 -0.0111 -0.0038 -0.0024 -0.0024  0.0012  0.00008
14894  20220831 -0.0074  0.0022 -0.0044 -0.0063 -0.0012  0.00008

[14895 rows x 7 columns]


This file is from http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html#Research and is created by CMPT_ME_BEME_OP_INV_RETS_DAILY using the 202208 CRSP database. The 1-month TBill return is from Ibbotson and Associates. For more details on the factor returns, please read http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_5_factors_2x3.html

In [54]:
# Inspect the dataframe with info()
factor_returns.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14895 entries, 19630701 to 20220831
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Mkt-RF  14895 non-null  float64
 1   SMB     14895 non-null  float64
 2   HML     14895 non-null  float64
 3   RMW     14895 non-null  float64
 4   CMA     14895 non-null  float64
 5   RF      14895 non-null  float64
dtypes: float64(6)
memory usage: 814.6 KB


In [65]:
# Inspect the last five observations

factor_returns.tail(5)
factor_returns.index = pd.to_datetime(factor_returns.index)

In [68]:
# Merge portfolio daily returns with factor returns
pd.concat([portfolio_returns, factor_returns[dt.datetime(2017,8,31):]], axis = 1)

,Equal_Weight_Return,MaxSharpe_Return,Minvol_Return,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,,,,
2017-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-09-01,-0.001321,0.004394,0.000662,NaN,NaN,NaN,NaN,NaN,NaN
2017-09-05,-0.003501,-0.000136,-0.000513,NaN,NaN,NaN,NaN,NaN,NaN
2017-09-06,-0.001780,-0.002856,-0.001092,NaN,NaN,NaN,NaN,NaN,NaN
2017-09-07,0.003493,0.001432,0.002346,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2022-08-25,0.010632,0.011389,0.008686,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-26,-0.028264,-0.032142,-0.027499,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-29,-0.004806,-0.005425,-0.004553,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
#factor_returns['Date'] = pd.to_datetime(factor_returns['Date'], format='%Y-%m-%d')
#factor_returns = factor_returns.set_index('Date')


DatetimeIndex(['1970-01-01 00:00:00.019630701',
               '1970-01-01 00:00:00.019630702',
               '1970-01-01 00:00:00.019630703',
               '1970-01-01 00:00:00.019630705',
               '1970-01-01 00:00:00.019630708',
               '1970-01-01 00:00:00.019630709',
               '1970-01-01 00:00:00.019630710',
               '1970-01-01 00:00:00.019630711',
               '1970-01-01 00:00:00.019630712',
               '1970-01-01 00:00:00.019630715',
               ...
               '1970-01-01 00:00:00.020220818',
               '1970-01-01 00:00:00.020220819',
               '1970-01-01 00:00:00.020220822',
               '1970-01-01 00:00:00.020220823',
               '1970-01-01 00:00:00.020220824',
               '1970-01-01 00:00:00.020220825',
               '1970-01-01 00:00:00.020220826',
               '1970-01-01 00:00:00.020220829',
               '1970-01-01 00:00:00.020220830',
               '1970-01-01 00:00:00.020220831'],
              dtype=

In [35]:
# Inspect the last five observations


In [36]:
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']
portfolio_return = ['portfolio_ew', 'portfolio_maxsharpe', 'portfolio_minvol']

# Print correlation table. Hint use .corr()


### 2. Examine visually the correlation between portfolio and factor returns

#### Hint: Write functions for repetitive lines of codes 

In [37]:
# function that takes portfolio and factor returns and creates 20-day rolling correlation with a return factor
def portfolio_factor_correlation(portfolio_type, df):
    
    ''' 
    INPUTS
    portfolio type: portfolio_ew, portfolio_maxsharpe, portfolio_minvol
    df: dataframe containing portfolio and factor returns
    
    OUTPUTS
    plot the 20-day rolling correlation between a portfolio and a factor return
    
    '''    
    
    for factor in factors:
        
        # Calculate 20-day rolling correlation with the market


        # Plot the correlation between a portfolio and factor returns


    return 

IndentationError: expected an indented block (4268343362.py, line 22)

In [ ]:
# Call the function portfolio_factor_correlation for the equally weighted portfolio


In [ ]:
# Call the function portfolio_factor_correlation for the maximum Sharpe portfolio


In [ ]:
# Call the function portfolio_factor_correlation for the minimum volatility portfolio


### 3. Regress the portfolio return on each factor and assess the portfolio's sensitivity to each factor.

#### For the curious, optional challenge, how do you test whether the intercept (i.e., alpha) is significantly different from the risk-free rate for a single-factor regression?
#### Hint: Write functions for repetitive lines of codes

In [ ]:
# Import the ols function
import statsmodels.api as sm

# function that takes portfolio and factor returns and run a regression of portfolio return on a factor return
# it reports the portfolio sensitivity to a return factor
def portfolio_factor_sensitivity(portfolio_type, df):
    
    ''' 
    INPUTS
    portfolio type: portfolio_ew, portfolio_maxsharpe, portfolio_minvol
    df: dataframe containing portfolio and factor returns
    
    OUTPUTS
    regression result of a portfolio on a return factor
    
    '''    
    for factor in factors:
        
        # Create the regression model object

        # Fit the model

        # Print the parameters of the fitted model

        # Optional challenge: testing the hypothesis that the intercept is significantly different from the risk-free rate
        # Hint: F-test

        
    return 

In [ ]:
# Call the function portfolio_factor_sensitivity for the equally weighted portfolio


In [ ]:
# Call the function portfolio_factor_sensitivity for the maximum Sharpe portfolio


In [ ]:
# Call the function portfolio_factor_sensitivity for minimum volatility portfolio


### 4. Regress the portfolio return on all factors and assess the portfolio's sensitivity to factors.

#### For the curious, optional challenge, how do you test whether the intercept (i.e., alpha) is significantly different from the risk-free rate for a multi-factor regression?
#### Hint: Write functions for repetitive lines of codes

In [ ]:
# Import the ols function
import statsmodels.api as sm

# function that takes portfolio and factor returns and run a regression of portfolio return on a return factor
# it reports the portfolio sensitivity to a return factor
def portfolio_all_factor_sensitivity(portfolio_type, df):
    
    ''' 
    INPUTS
    portfolio type: portfolio_ew, portfolio_maxsharpe, portfolio_minvol
    df: dataframe containing portfolio and factor returns
    
    OUTPUTS
    regression result of a portfolio on a return factor
    
    '''    
        
    # Create the model object

    
    # Fit the model

    # Print the parameters of the fitted model
    
    # Optional challenge: testing the hypothesis that the intercept is significantly different from the risk-free rate
    # Hint: F-test


    return 

In [ ]:
# Call the function portfolio_all_factor_sensitivity for the equally weighted portfolio


In [ ]:
# Call the function portfolio_all_factor_sensitivity for the maximum Sharpe portfolio


In [ ]:
# Call the function portfolio_all_factor_sensitivity for the minimum volatility portfolio


### 5. Optional Bonus. Construct a multi-factor pricing model for assets based on Arbitrage Pricing Theory.

#### The Arbitrage Pricing Theory (APT) is a theory of asset pricing that holds that an asset’s returns can be forecasted with the linear relationship of an asset’s expected returns and the macroeconomic (e.g., GDP, changes in inflation, yield curve changes, changes in interest rates, market sentiments, exchange rates) or firm-specific statistical factors that affect the asset’s risk. Hint: You can draw these variables straight into your Jupyter notebook via Refinitiv API.

#### The APT is a substitute for the Capital Asset Pricing Model (CAPM) in that both assert a linear relation between assets’ expected returns and their covariance with other random variables. (In the CAPM, the covariance is with the market portfolio’s return.) The covariance is interpreted as a measure of risk that investors cannot avoid by diversification. The slope coefficient in the linear relation between the expected returns and the covariance is interpreted as a risk premium ~ "Arbitrage Pricing Theory (Guberman and Wang 2005).

Acknowledgement: This notebook is inspired by DataCamp course "Introduction to Portfolio Analysis in Python" by Charlotte Werger.